#### 【 데이터 전처리 - 표준화(Standardization) 수치형 피쳐 】
- 피쳐 스케일링
    * 변수 간 단위와 크기 차이 제거
    * 모델 학습 안정화 및 수렴 속도 향상
    * 가중치 비교 가능하게 만듦
    * 거리 기반 알고리즘(KNN, SVM 등)에서 공정한 거리 계산
- 종류
    * 표준화(StandardScaler) : 분포형태 유지.중심과 분산만 조정하는 스케일링 기법
        - > Z-score Standardization (기본 표준화) : 평균 μ, 표준편차 σ 기준으로 변환 평균0, 표준편차 1
        - > Mean Normalization (평균 기준 스케일링) : 표준편차가 아닌 범위 사용하는 중심 정렬형 스케일링. -1 ~ 1
        - > Robust Standardization (RobustScaler) : 중앙값(Median)과 IQR(Interquartile Range) 사용
        - > MaxAbs Standardization : 절댓값 기준으로 분포를 -1~1 범위로

    * 정규화 (Normalization) : 데이터의 절대 크기(범위) 를 일정한 구간 안으로 맞추는 스케일링
        - > Min–Max Scaling : 0 ~ 1
        - > Max–Abs Scaling : -1 ~ 1
        - > L1 / L2 Normalization : 0 ~ 1

    * 하이브리드 스케일러 : 표준화(Standardization) 와 정규화(Normalization) 의 특징이 혼합된 스케일링 방법
        - > MaxAbsScaler : 중심 유지, 분포 형태 유지, 값 범위 -1 ~ 1
        - > RubustScaler : 중심과 분포 유지, IQR(사분위 범위, Q3–Q1)로 나눔. 정규화의 스케일 안정성 확보
        - > PowerTransformer :비정규 분포를 정규분포에 가깝게 조정하는 비선형 표준화형
        - > QuantileTransformer(분위수 정규화형) : 분포를 일정 기준(균등·정규)으로 보정, 값의 범위가 0~1로 제한됨

[1] 모듈 로딩 및 데이터 준비<hr>

In [55]:
## 모듈 로딩 
import pandas as pd 
import numpy as np
import sklearn                                                  ## ML 전용 패키지
from sklearn.preprocessing import MinMaxScaler, StandardScaler  ## ML의 전처리 서브모듈의 스케일러
from sklearn.preprocessing import RobustScaler

In [56]:
## 데이터 준비
DATA_FILE = '../Data/auto_mpg.csv' 

##-> csv 파일 경우 : 첫 번째 줄 컬럼명 여부 체크,  데이터 구분자 쉼표(,) 체크
mpgDF = pd.read_csv(DATA_FILE)    

## 데이터 확인
mpgDF.info()                                            ## 기본 요약 정보
display(mpgDF.head(3), mpgDF.describe(include='all'))   ## 실제 데이터, 컬럼별 통계 데이터 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
count,398.000000,398.000000,398.000000,398,398.000000,398.000000,398.000000,398.000000,398
unique,NaN,NaN,NaN,94,NaN,NaN,NaN,NaN,305
top,NaN,NaN,NaN,150,NaN,NaN,NaN,NaN,ford pinto
freq,NaN,NaN,NaN,22,NaN,NaN,NaN,NaN,6
mean,23.514573,5.454774,193.425879,NaN,2970.424623,15.568090,76.010050,1.572864,NaN
std,7.815984,1.701004,104.269838,NaN,846.841774,2.757689,3.697627,0.802055,NaN
min,9.000000,3.000000,68.000000,NaN,1613.000000,8.000000,70.000000,1.000000,NaN
25%,17.500000,4.000000,104.250000,NaN,2223.750000,13.825000,73.000000,1.000000,NaN
50%,23.000000,4.000000,148.500000,NaN,2803.500000,15.500000,76.000000,1.000000,NaN
75%,29.000000,8.000000,262.000000,NaN,3608.000000,17.175000,79.000000,2.000000,NaN


In [57]:
##-> 수치형 피쳐만 선택
mpgDF2 = mpgDF.select_dtypes(include="number")

mpgDF2.head(3)

,mpg,cylinders,displacement,weight,acceleration,model year,origin
0,18.0,8,307.0,3504,12.0,70,1
1,15.0,8,350.0,3693,11.5,70,1
2,18.0,8,318.0,3436,11.0,70,1


In [58]:


# 표준화 
def zscore_scale(X):
    mu = X.mean()
    sigma = X.std(ddof=0)
    return (X - mu) / sigma

def minmax_scale(X):  ## 얘는 많이 안씀
    xmin = X.min()
    xmax = X.max()
    rng = xmax - xmin
    # Avoid division by zero: where rng==0, leave as 0
    return (X - xmin).divide(rng.replace(0, pd.NA)).fillna(0)

def maxabs_scale(X):
    m = X.abs().max()
    return X.divide(m.replace(0, pd.NA)).fillna(0) ## 결측치 값은 빼버리기 때문에 결측치로 치환하는거임

def robust_scale(X):
    med = X.median()
    q1 = X.quantile(0.25) ## 하위 25%
    q3 = X.quantile(0.75) ## 상위 75%
    iqr = q3 - q1
    iqr = iqr if iqr else 1
    return (X - med).divide(iqr)


In [59]:
x_med = mpgDF2['mpg'] - mpgDF2['mpg'].median()

q1 = mpgDF2['mpg'].quantile(0.25) ## 하위 25%
q3 = mpgDF2['mpg'].quantile(0.75) ## 상위 75%
q1, q3, q3 - q1

(np.float64(17.5), np.float64(29.0), np.float64(11.5))

In [60]:
robust_scale(mpgDF2['mpg'])  ## 중앙값을 기반으로 스케일링 진행

0     -0.434783
1     -0.695652
2     -0.434783
3     -0.608696
4     -0.521739
         ...   
393    0.347826
394    1.826087
395    0.782609
396    0.434783
397    0.695652
Name: mpg, Length: 398, dtype: float64

In [61]:
zscore_scale(mpgDF2['mpg'])   ## 범위 맞출려고 쓰는거임

0     -0.706439
1     -1.090751
2     -0.706439
3     -0.962647
4     -0.834543
         ...   
393    0.446497
394    2.624265
395    1.087017
396    0.574601
397    0.958913
Name: mpg, Length: 398, dtype: float64

In [62]:
zscore_scale(mpgDF2).describe()

,mpg,cylinders,displacement,weight,acceleration,model year,origin
count,3.980000e+02,3.980000e+02,3.980000e+02,3.980000e+02,3.980000e+02,3.980000e+02,3.980000e+02
mean,7.141133e-17,-3.570567e-17,-1.785283e-17,-1.606755e-16,-2.677925e-16,-1.642461e-15,-5.355850e-17
std,1.001259e+00,1.001259e+00,1.001259e+00,1.001259e+00,1.001259e+00,1.001259e+00,1.001259e+00
min,-1.859374e+00,-1.444949e+00,-1.204411e+00,-1.604943e+00,-2.747814e+00,-1.627426e+00,-7.151448e-01
25%,-7.704907e-01,-8.563206e-01,-8.563178e-01,-8.828266e-01,-6.328794e-01,-8.150739e-01,-7.151448e-01
50%,-6.591883e-02,-8.563206e-01,-4.314040e-01,-1.973624e-01,-2.472221e-02,-2.721449e-03,-7.151448e-01
75%,7.027050e-01,1.498191e+00,6.584879e-01,7.538337e-01,5.834349e-01,8.096310e-01,5.332220e-01
max,2.957335e+00,1.498191e+00,2.511784e+00,2.565185e+00,3.351912e+00,1.621983e+00,1.781589e+00


[2] 머신러닝 패키지의 전처리 서브모듈의 스케일러 활용 <hr>

In [63]:
## MinMaxScaler로 피쳐 값 0 ~ 1 범위로 맞추기
## [1] Pandas 기반
min_v = mpgDF['mpg'].min()
max_v = mpgDF['mpg'].max()

## 공식 : (data-min)/(max-min)
minmaxSR = (mpgDF['mpg'] - min_v) / (max_v - min_v)
minmaxSR

0      0.239362
1      0.159574
2      0.239362
3      0.186170
4      0.212766
         ...   
393    0.478723
394    0.930851
395    0.611702
396    0.505319
397    0.585106
Name: mpg, Length: 398, dtype: float64

In [64]:
## [2] Scikit-learn 기반  -> 2차원을 줘야함
mmScaler = MinMaxScaler()
mpgDF['mm_mpg'] = mmScaler.fit_transform(mpgDF[['mpg']])

mpgDF.head(2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,mm_mpg
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu,0.239362
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320,0.159574


In [65]:
## ------------------------------------------
## StandardScaler : 평균 0, 표준편차 1
## ------------------------------------------
## [1] Pandas 기반
mean_v = mpgDF['mpg'].mean()
std_v = mpgDF['mpg'].std()

## 공식 : (data - mean)/(std)
stdSR = (mpgDF['mpg'] - mean_v) / (std_v)
stdSR 

0     -0.705551
1     -1.089379
2     -0.705551
3     -0.961437
4     -0.833494
         ...   
393    0.445936
394    2.620966
395    1.085650
396    0.573879
397    0.957708
Name: mpg, Length: 398, dtype: float64

In [66]:
## [2] Scikit-learn 기반  -> 2차원을 줘야함
ssScaler = StandardScaler()
mpgDF['ss_mpg'] = ssScaler.fit_transform(mpgDF[['mpg']])

mpgDF.head(2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,mm_mpg,ss_mpg
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu,0.239362,-0.706439
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320,0.159574,-1.090751
